
1. **Définir le problème**
 

2. Récupérer les données
 

3. Analyser et nettoyer les données
  

4. Préparer les données
  

5. Evaluer plusieurs modèles
  

6. Réglage fin des modèles
 

7. Surveiller son modèle







## 📌 1. Définir le problème

## **Exemple : Prédiction des résultats des contrôles qualité patients VMAT en radiothérapie**

L'exemple présenté repose sur des données issues de plans de traitement VMAT. Cependant, la méthodologie décrite s'applique à toute autre source de données tabulées.

### 1.1 Objectif
<img src="https://physik.gesundheitsverbund.at/img30/Donald22.png" alt="drawing" width="300"/>


- **Contexte** : Les plans de traitement sont contrôlés après validation médicale via le logiciel Portal Dosimetry de Varian.
- **Problème** : Le CQ patient est donc réalisé peu de temps avant le début programmé du traitement, ce qui laisse peu ou pas de **marge d'évaluation et/ou de correction** si un plan est hors tolérance ou un résultat inhabituel.
- **Objectif** : **Prédire les CQ inhabituels pour permettre une évaluation/correction avant le début du traitement.**

### 1.2 Contraintes
<img src="https://www.oregoncancer.com/hs-fs/hubfs/radiation-oncologist-team-wvci.jpg?width=545&height=442&name=radiation-oncologist-team-wvci.jpg" alt="drawing" width="220"/>

- **Utilisateurs** : Le modèle sera utilisé par les **dosimétristes**.
- **Intégration** : Interface utilisateur simple et intuitive, avec **temps de calcul rapide** et **peu d’interactions manuelles** (import/export de données, nombre de clics, etc).

### 1.3 Solutions actuelles
<img src="https://ekamperi.github.io/mrm_thesis/images/plan_irreg_schematic.PNG" alt="drawing" width="200"/>

- Corrélations directes entre métriques de complexité et résultats de CQ : souvent les corrélations sont assez faibles.
- Comparaison entre cette approche simple et un modèle prédictif plus avancé.

### 1.4 Formulation du problème
<img src="https://images.ctfassets.net/txbhe1wabmyx/7aytfXgoS6vadz4iGzZWWX/f08c78f400ddcd3da343c3f569d03c43/pexels-pixabay-355948-1.jpg" alt="drawing" width="200"/>

- **Cibles** : Résultats des CQ patients (présents dans la base ARIA).
- **Nature du problème** : **Classification supervisée (pass/fail).**
- **Variables utilisées** : Métriques de complexité calculées pour chaque plan (collecte automatisée via un script C# qui parcourt toute la base de données ARIA et calcule les métriques à partir des paramètres des arcs VMAT).

### 1.5 Performance recherchée
<img src="https://www.atelierdespratiques.fr/sites/atelierdespratiques.fr/files/2022-05/Indicateurs%20de%20performance.jpg" alt="drawing" width="150"/>

- **Classe positive** : Plans hors tolérance (*fail*).
- **Sensibilité minimale** : ≥ 50% (détecter au moins 30% des échecs).
- **Spécificité minimale** : ≥ 90% (éviter trop de fausses alertes).
- **Évaluation** : 
  - **Courbes ROC** (TPR vs. FPR).
  - **Courbes précision-rappel** (PPV vs. TPR).
  - **Matrice de confusion**.
- **Attention au déséquilibre des classes !**
  Avec un déséquilibre 90/10, un modèle naif qui détecte tous les CQ commes des "pass" a une spécificité et une accuracy de 90%.  


> ⚠ **L'accuracy seule n'est pas un bon indicateur de performance dans un contexte de classes déséquilibrées. L'accuracy est souvent utilisée par défaut comme estimateur de performance dans les fonctions Scikit de python**



### 📊 Métriques de classification

Les métriques suivantes sont utilisées pour évaluer la qualité d'un modèle de classification binaire basé sur ses prédictions.

#### 🔢 Notations :
- **TP (True Positives)** : Nombre de cas positifs (fail) correctement détectés.
- **TN (True Negatives)** : Nombre de cas négatifs (pass) correctement détectés.
- **FP (False Positives)** : Nombre de cas négatifs (pass) incorrectement détectés comme positifs (fail).
- **FN (False Negatives)** : Nombre de cas positifs (fail) incorrectement détectés comme négatifs (pass).
- **P (Positifs réels)** : $ P = TP + FN $ (*total des vrais positifs (fail)*).
- **N (Négatifs réels)** : $ N = TN + FP $ (*total des vrais négatifs (pass)*).

---

### 📈 Formules des métriques

| **Métrique**                          | **Formule**                          | **Interprétation** |
|----------------------------------------|:--------------------------------------:|--------------------|
| **Probabilité de fausse alarme** (*False Positive Rate - FPR*) | $$ \text{FPR} = \frac{FP}{N} = 1 - \text{TNR} $$ | Proportion de cas négatifs incorrectement détectés comme positifs. |
| **Taux d'échec** (*False Negative Rate - FNR*) | $$ \text{FNR} = \frac{FN}{P} = 1 - \text{TPR} $$ | Proportion de cas positifs incorrectement détectés comme négatifs. |
| **Sensibilité** (*Rappel / True Positive Rate - TPR*) | $$ \text{TPR} = \frac{TP}{P} = 1 - \text{FNR} $$ | Proportion de cas positifs correctement détectés. |
| **Spécificité** (*True Negative Rate - TNR*) | $$ \text{TNR} = \frac{TN}{N} = 1 - \text{FPR} $$ | Proportion de cas négatifs correctement détectés. |
| **Précision** (*Positive Predictive Value - PPV*) | $$ \text{PPV} = \frac{TP}{TP + FP} = 1 - \text{FDR} $$ | Proportion de cas positifs correctement détectés parmi toutes les détections positives. |
| **Taux de fausses découvertes** (*False Discovery Rate - FDR*) | $$ \text{FDR} = \frac{FP}{TP + FP} = 1 - \text{PPV} $$ | Proportion de fausses détections parmi toutes les détections positives. |
| **Exactitude** (*Accuracy*) | $$ \text{Accuracy} = \frac{TP + TN}{P + N} $$ | Proportion de prédictions correctes parmi l’ensemble des cas. |

---

### 📌 Remarque :
- Un bon modèle a une **sensibilité élevée ($\text{TPR}$)** et une **spécificité élevée ($\text{TNR}$)**.  
- La **précision ($\text{PPV}$)** est importante lorsque le coût des fausses alertes ($FP$) est élevé.  

---

